# AI-Enhanced Inventory Optimization — Answer Key

In [ ]:
import pandas as pd, numpy as np
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

df = pd.read_csv("../data/daily_demand.csv", parse_dates=["date"]).sort_values("date")
# Build simple lag features
for lag in [1,2,3,7]:
    df[f"lag{lag}"] = df["demand"].shift(lag)
df = df.dropna().reset_index(drop=True)
split = int(len(df)*0.8)
train, test = df.iloc[:split], df.iloc[split:]

X_train = train[[f"lag{l}" for l in [1,2,3,7]]].values
y_train = train["demand"].values
X_test = test[[f"lag{l}" for l in [1,2,3,7]]].values
y_test = test["demand"].values

rf = RandomForestRegressor(random_state=42).fit(X_train, y_train)
test["forecast"] = rf.predict(X_test)

# Inventory parameters
lead_time_days = 5
service_factor = 1.65
demand_std = train["demand"].rolling(30).std().dropna().mean()
daily_mu = test["forecast"]
K = 100.0; h = 1.0
annual_demand = df["demand"].sum()
EOQ = ((2*K*annual_demand)/h)**0.5

mu_LT = daily_mu.rolling(lead_time_days).sum().fillna(daily_mu.mean()*lead_time_days)
sigma_LT = np.sqrt(lead_time_days) * demand_std
safety_stock = service_factor * sigma_LT
ROP = mu_LT + safety_stock

print("EOQ:", round(EOQ,2))
out = pd.DataFrame({"date": test["date"], "forecast": daily_mu, "ROP": ROP})
display(out.head())

plt.figure()
plt.plot(out["date"], out["forecast"], label="forecast")
plt.plot(out["date"], out["ROP"], label="ROP")
plt.title("Forecast & Reorder Point")
plt.legend()
plt.show()


## Monte Carlo: AI vs Static Policy

In [ ]:
import numpy as np, pandas as pd
np.random.seed(0)
horizon=200
demand = np.random.poisson(40, size=horizon)
def simulate(policy, reorder_qty=200, initial=100):
    inv=initial; stockouts=0
    for t in range(horizon):
        if policy(t, inv): inv += reorder_qty  # instant repl. for simplicity
        if inv>=demand[t]: inv -= demand[t]
        else:
            stockouts += (demand[t]-inv); inv=0
    return stockouts

# Static policy: reorder when inv<150
static = simulate(lambda t,inv: inv<150)
# 'AI' policy: more sensitive when recent demand spike
recent = np.convolve(demand, np.ones(7)/7, mode="same")
def ai_policy(t, inv): 
    thresh = 150 + (recent[t]-40)*2
    return inv < thresh
ai = simulate(ai_policy)
print("Stockouts - Static:", static, " | AI:", ai)
